In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("./data/pred_objs_total_opt1.csv")
df.columns

Index(['org_utterance', 'object', 'relation', 'subject', 'id'], dtype='object')

In [ ]:
df2 = pd.read_csv("./data/nr3d.csv")

In [ ]:
res_relations = []
for i in range(df.shape[0]):
    res_relations.append({"org_utterance": df.utterance[i].lower(),
                                                "object": df.object[0],
                                                "relation"        : objects_relation_pred[1],
                                                "subject": objects_relation_pred[2],
                                                "id": i
                                                })

In [3]:
df = pd.read_csv('total_correct.csv')
df.shape

(5521, 61)

In [4]:
SINGULAR_UNINFLECTED = ['gas', 'asbestos', 'womens', 'childrens', 'sales', 'physics']

SINGULAR_SUFFIX = [
    ('people', 'person'),
    ('men', 'man'),
    ('wives', 'wife'),
    ('menus', 'menu'),
    ('us', 'us'),
    ('ss', 'ss'),
    ('is', 'is'),
    ("'s", "'s"),
    ('ies', 'y'),
    ('ies', 'y'),
    ('es', 'e'),
    ('s', '')
]
def singularize_word(word):
    for ending in SINGULAR_UNINFLECTED:
        if word.lower().endswith(ending):
            return word
    for suffix, singular_suffix in SINGULAR_SUFFIX:
        if word.endswith(suffix):
            return word[:-len(suffix)] + singular_suffix
    return word

In [12]:
from scannet_classes import SCANNET_OBJECTS
import string
from difflib import SequenceMatcher as SM
#Using apply function to get the class name
annoted_wrong = []
for i in range(df.shape[0]):
# for i in range(829,830):
    #Get each object
    for j in range(1,9):
        feature = "Answer.object" + str(j)
        new_feature = "Answer.object" + str(j) + "_preprocessed"
        object_ = str(df.loc[i,feature]).lower()
        object_ = object_.translate(str.maketrans('', '', string.punctuation))
        if object_ == "nan":
            continue
        if not(object_ in str(df['Input.utterance'].iloc[i]).lower()):
            # print("The annotation of the objects: ", object_, "is not right", "in the sentence: ", str(df['Input.utterance'].iloc[i]))
            annoted_wrong.append([df['HITId'].iloc[i], df['Input.utterance'].iloc[i], object_])
            if object_.split(" ")[-1] in str(df['Input.utterance'].iloc[i]).lower().split(" "):
                object_ = object_.split(" ")[-1]
            else:
                continue
                
             
        best_len = 10000
        
        '''
        1. Check for Exact Match. If so, we are done 
        2. If not, check for the longest match
        ''' 
        for obj in SCANNET_OBJECTS:
            if len(object_.split(" ")) == 1:
                object_ = singularize_word(object_)
            if object_ == 'nan':
                continue
            elif object_ == str(obj) or obj == str(object_):
                df.loc[i, new_feature] = obj
                break
            else:
                #Do substring matching
                annotated_object = object_.lower().split(" ")
                temp = annotated_object

                for idx, obj_ in enumerate(annotated_object):
                    temp[idx] = singularize_word(obj_)
                annotated_object = temp
                
                scannet_object = obj.lower().split(" ")
                temp = scannet_object
                for idx, obj_ in enumerate(scannet_object):
                    temp[idx] = singularize_word(obj_)
                scannet_object = temp
                
                if len(annotated_object) > len(scannet_object) : 
                    for left in range(len(scannet_object)):
                        for right in range(left, len(scannet_object)): 
                            if len(" ".join(scannet_object[left:right+1])) > 0 and " ".join(scannet_object[left:right+1]) in object_  and len(obj) <= best_len and len(obj) >= len(object_.split(" ")[-1]):
                                df.loc[i, new_feature] = obj
                                best_len = len(obj)
                else: #len(annoted_object) <= len(scannet_object)
                    for left in range(len(annotated_object)):
                        for right in range(left, len(annotated_object)): 
                            if len(" ".join(annotated_object[left:right+1])) > 0 and " ".join(annotated_object[left:right+1]) in obj and len(obj) <= best_len and len(obj) >= len(object_.split(" ")[-1]):
                                df.loc[i, new_feature] = obj
                                best_len = len(obj)
                                
                if new_feature not in df.columns:
                    #Do the greedy part 
                    last  = singularize_word(object_.split(" ")[-1].split('/')[-1].lower())
                    if last == obj:
                        df.loc[i, new_feature] = obj
                    else:
                        first = singularize_word(object_.split(" ")[0].split('/')[0].lower())
                        if first == obj:
                            df.loc[i, new_feature] = first
                    
                        
    

In [47]:
df_wrong = pd.DataFrame(annoted_wrong, columns=['HITId', 'Input.utterance', 'Answer.object'])

In [48]:
df_wrong.to_csv('annoted_wrong.csv', index=False)

In [49]:
5000*9*800

36000000

In [14]:
#add it in csv file
df.to_csv('total_correct_preprocessed_objects.csv')

In [16]:
df['Answer.object1_preprocessed'].iloc[829]

nan

In [38]:
import pandas as pd

RELATIONS = [
    'above',
    'behind',
    'below',
    'beneath',
    'beside',
    'between',
    'close to',
    'closer to',
    'far away from',
    'far from',
    'farthest from',
    'in front of',
    'in the center of',
    'in the middle of',
    'lying on',
    'near',
    'next to',
    'on',
    'on the back of',
    'on the left of',
    'on the left side of',
    'on the right of',
    'on the right side of',
    'on top',
    'on top of',
    'over',
    'supporting',
    'to the left of',
    'to the right of',
    'under',
    'underneath',
    'beside',
    'close to',
    'closer to',
    'near',
    'next to',
    'far away from',
    'far from',
    'farthest from',
    'between',
    'in the center of',
    'in the middle of',
    'behind',
    'in front of',
    'on the back of',
    'on the left of',
    'on the left side of',
    'on the right of',
    'on the right side of',
    'to the left of',
    'to the right of',
    'above',
    'below',
    'beneath',
    'lying on',
    'on',
    'on top',
    'on top of',
    'over',
    'supporting',
    'under',
    'underneath',
    'back',
    'front'
]

def relation_check(target_string, string_list):
    for str in RELATIONS:
        if any(substring in str for substring in target_string.split()):
            return False
    return True

# load the csv file into a pandas dataframe
df = pd.read_csv('/home/mohama0e/CoT3D_VG/automatic_loc_module/data/pred_objs_total_opt2.csv')
print(df.columns)
print(df.head())
# get the unique values of the 'relations' column
mask = df['relation'].apply(lambda x: relation_check(x, RELATIONS))

# filter the dataframe using the mask
filtered_df = df[mask]

#Get Unique relation 
unique_relation = filtered_df['relation'].unique()

# write the filtered dataframe to a new CSV file


Index(['org_utterance', 'pred_obj_name_1', 'relation', 'pred_obj_name_2'], dtype='object')
                                       org_utterance pred_obj_name_1   
0  the plant at the far right hand side of the bo...        bookcase  \
1  the plant at the far right hand side of the bo...            desk   
2  the towel hanging draped over the side of the ...             tub   
3     the short trashcan that is closest to the door            door   
4  the pillow on the couch that is on the right h...           couch   

                        relation pred_obj_name_2  
0  at the far right hand side of           plant  
1      in the furthest corner of        bookcase  
2               over the side of   towel hanging  
3                     closest to        trashcan  
4                             on          pillow  


In [39]:
len(unique_relation)

979

In [ ]:
filtered_df.to_csv('output_file_opt1.csv', index=False)

In [2]:
import pandas as pd

RELATIONS = [
    'above',
    'behind',
    'below',
    'beneath',
    'beside',
    'between',
    'close to',
    'closer to',
    'far away from',
    'far from',
    'farthest from',
    'in front of',
    'in the center of',
    'in the middle of',
    'lying on',
    'near',
    'next to',
    'on',
    'on the back of',
    'on the left of',
    'on the left side of',
    'on the right of',
    'on the right side of',
    'on top',
    'on top of',
    'over',
    'supporting',
    'to the left of',
    'to the right of',
    'under',
    'underneath',
    'beside',
    'close to',
    'closer to',
    'near',
    'next to',
    'far away from',
    'far from',
    'farthest from',
    'between',
    'in the center of',
    'in the middle of',
    'behind',
    'in front of',
    'on the back of',
    'on the left of',
    'on the left side of',
    'on the right of',
    'on the right side of',
    'to the left of',
    'to the right of',
    'below',
    'beneath',
    'lying on',
    'on',
    'on top',
    'on top of',
    'over',
    'supporting',
    'under',
    'underneath',
    'back',
    'front',
    'facing',
    'with',
    'touching',
    'tucked',
    'draped',
    'facing',
    'with',
    'grouped',
    'touching',
    'along',
    'against',
    'through',
    'with',
    'near',
    'opposite',
    'by',
    'closest',
    'down',
    'face',
    'atop',
    'leftmost',
    'along',
    'overlooks',
    'looking',
    'off',
    'across',
    'nearest',
    'towards',
    'faces',
    'higher',
    'beaneath',
    'inbewteen',
    'furthest',
    'besides',
    'nearer',
    'topped',
    'within',
    'alongside',
    'mounted', 
    'containing', 
    'covering', 
    'contain', 
    'contains', 
    'around', 
    'inside', 
    'sits', 
    'lower',
    'than', 
    'among', 
    'overhangs', 
    'holding', 
    'includes', 
    'blocking', 
    'before', 
    'sitting', 
    'block', 
    'laying', 
    'seats', 
    'onto', 
    'taped', 
    'covers', 
    'hangs', 
    'covered', 
    'after', 
    'cover', 
    'include', 
    'beyond', 
    'setting', 
    'touched',
    'inbetween',
    'surrounds',
    'nearby',

]

def relation_check(target_string, string_list):
    for str in RELATIONS:
        if any(substring.lower() in str.lower() for substring in target_string.lower().split()):
            return True
    return False

# load the csv file into a pandas dataframe
df = pd.read_csv('data/pred_objs_total_opt2.csv')
nr_df = pd.read_csv('data/nr3d.csv')

print(df.columns)
print(df.head())
# get the unique values of the 'relations' column
mask = df['relation'].apply(lambda x: relation_check(x, RELATIONS))

# filter the dataframe using the mask
filtered_df = df[mask]
unique_relation = filtered_df['relation'].unique()

#Add unique relation in a csv file 

unique_relation_df = pd.DataFrame(unique_relation)

unique_relation_df.to_csv('unique_relation.csv', index=False)

# filtered_df['id'] = filtered_df['id'].astype(int)

# filtered_nr_df = nr_df.iloc[filtered_df['id'].unique()]

# sampled_df = filtered_nr_df.sample(n=4000, random_state=42)
# sampled_df = sampled_df.rename(columns={'assignmentid': 'id'})
# # write the filtered dataframe to a new CSV file
# filtered_df.to_csv('spacial_relations.csv', index=False)
# filtered_nr_df.to_csv('filtered_nr.csv', index=False)
# sampled_df.to_csv('filtered_nr4k.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/pred_objs_total_opt2.csv'

In [51]:
df2 = pd.read_csv('data/pred_objs_total_opt1.csv')
nr3d_df = pd.read_csv('data/nr3d.csv')




In [54]:
df2.columns

Index(['org_utterance', 'pred_obj_name_1', 'relation', 'pred_obj_name_2',
       'id'],
      dtype='object')

In [65]:
pronouns = ['it', 'they', 'them', 'he', 'she', 'him', 'her', 'his', 'hers', 'its', 'their', 'theirs', 'himself', 'herself', 'themselves']

In [68]:
df2.shape

(52764, 5)

In [69]:
nr3d_df.shape

(41503, 16)

In [70]:
df2['id']

0            0
1            0
2            1
3            2
4            3
         ...  
52759    41499
52760    41500
52761    41501
52762    41502
52763    41502
Name: id, Length: 52764, dtype: int64

In [71]:
for idx in range(df2.shape[0]):
    id = df2['id'].iloc[idx]
    if str(df2['pred_obj_name_1'].iloc[idx]) in pronouns:
        df2['pred_obj_name_1'].iloc[idx] =  nr3d_df.iloc[id]['instance_type']
    if str(df2['pred_obj_name_2'].iloc[idx]) in pronouns:
        df2['pred_obj_name_2'].iloc[idx] =  nr3d_df.iloc[id]['instance_type']

/tmp/ipykernel_42845/2240542550.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pred_obj_name_2'].iloc[idx] =  nr3d_df.iloc[id]['instance_type']
/tmp/ipykernel_42845/2240542550.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pred_obj_name_1'].iloc[idx] =  nr3d_df.iloc[id]['instance_type']
/tmp/ipykernel_42845/2240542550.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['pred_obj_name_2'].iloc[idx] =  nr3d_df.ilo

In [73]:
#Create a new csv file with the updated pronouns: with features "id", "org_utterance", "object", "subject"

new_df = df2[['id', 'org_utterance', 'pred_obj_name_2', 'pred_obj_name_1']]

In [75]:
#Rename the columns
new_df = new_df.rename(columns={'pred_obj_name_2': 'object', 'pred_obj_name_1': 'subject'})

In [76]:
#CSV
new_df.to_csv('data/updated_pronouns_opt1.csv', index=False)

In [61]:
nr3d_df.iloc[0]

assignmentid                                                         32618
stimulus_id                             scene0525_00-plant-5-9-10-11-12-62
utterance                The plant at the far right hand side of the bo...
correct_guess                                                         True
speaker_id                                                              47
listener_id                                                            310
scan_id                                                       scene0525_00
instance_type                                                        plant
target_id                                                                9
tokens                   ['the', 'plant', 'at', 'the', 'far', 'right', ...
dataset                                                               nr3d
mentions_target_class                                                 True
uses_object_lang                                                      True
uses_spatial_lang        

In [58]:
new_df.iloc[0]

org_utterance            the plant at the far right hand side of the bo...
pred_obj_name_1                                                   bookcase
relation                                     at the far right hand side of
pred_obj_name_2                                                      plant
id                                                                       0
assignmentid                                                           NaN
stimulus_id                                                            NaN
utterance                                                              NaN
correct_guess                                                          NaN
speaker_id                                                             NaN
listener_id                                                            NaN
scan_id                                                                NaN
instance_type                                                          NaN
target_id                

In [53]:
nr3d_df.columns

Index(['assignmentid', 'stimulus_id', 'utterance', 'correct_guess',
       'speaker_id', 'listener_id', 'scan_id', 'instance_type', 'target_id',
       'tokens', 'dataset', 'mentions_target_class', 'uses_object_lang',
       'uses_spatial_lang', 'uses_color_lang', 'uses_shape_lang'],
      dtype='object')